# 🚀 Universal LLM Server for OpenCode

**Change the MODEL variable below to use any Ollama model.**

Recommended models:
- `deepseek-coder-v2:16b` - Best coding (current)
- `qwen2.5-coder:7b` - Fast, good coding
- `codellama:13b` - Code generation
- `mistral:7b` - General purpose

In [ ]:
#@title ⚙️ Configuration
MODEL = "deepseek-coder-v2:16b"  #@param {type:"string"}
print(f"📦 Selected model: {MODEL}")

In [ ]:
#@title 📥 Install Dependencies
!nvidia-smi
!curl -fsSL https://ollama.com/install.sh | sh
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!pip install -q flask requests
print('\n✅ Dependencies installed')

In [ ]:
#@title 🤖 Start Ollama & Download Model
import subprocess, time, os
os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
os.environ['OLLAMA_ORIGINS'] = '*'
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print(f'Downloading {MODEL}... (this may take a while)')
!ollama pull {MODEL}
print(f'\n✅ {MODEL} ready!')

In [ ]:
#@title 🔧 OpenCode-Compatible API Server
from flask import Flask, request, jsonify
import requests as req
import json, time, uuid, threading, re

app = Flask(__name__)

# ═══════════════════════════════════════════════════════════════
# SYSTEM PROMPT - Elite Coding Assistant
# ═══════════════════════════════════════════════════════════════
SYSTEM_PROMPT = '''You are an elite AI coding assistant. You are precise, efficient, and follow best practices.

## How to Respond

### For Greetings & Questions (NO tools)
- Respond naturally and helpfully
- Be concise but informative
- Provide code examples in markdown when helpful

### For File Operations (USE tools)
When asked to create, edit, read, or write files, use the provided tools.
Format tool calls as JSON:
```json
{"name": "tool_name", "arguments": {"param": "value"}}
```

## Code Standards
- Write clean, production-ready code
- Use meaningful names and add comments for complex logic
- Handle errors properly
- Follow language conventions (PEP8, ESLint, etc)

## Important Rules
1. ONLY use tools when explicitly asked to perform file/system operations
2. For questions about code, respond with explanations and examples
3. Be concise - don't over-explain
4. If unsure, ask for clarification'''

# ═══════════════════════════════════════════════════════════════
# Helper Functions
# ═══════════════════════════════════════════════════════════════

def needs_tools(msg):
    """Detect if message requires tool usage."""
    m = msg.lower()
    actions = ['create', 'write', 'make', 'generate', 'add', 'new', 'save',
               'edit', 'modify', 'update', 'change', 'fix', 'refactor', 'rename',
               'delete', 'remove', 'read', 'open', 'show file', 'cat ', 'view file',
               'run', 'execute', 'test', 'build', 'deploy', 'install',
               'search', 'find', 'grep', 'locate', 'list files',
               'file', 'folder', 'directory', 'path',
               'todo', 'task', 'bash', 'terminal', 'command', 'shell']
    return any(a in m for a in actions)

def get_valid_tools(tools):
    """Extract tool names from definitions."""
    return {t['function']['name'] for t in tools or [] if t.get('type') == 'function' and 'function' in t}

def extract_tool(text, valid):
    """Extract tool call from response text."""
    if not text or not valid:
        return None
    
    # Try code block
    m = re.search(r'```(?:json)?\s*([\s\S]*?)```', text)
    if m:
        try:
            d = json.loads(m.group(1).strip())
            if d.get('name') in valid:
                return d['name'], json.dumps(d.get('arguments', {}))
        except: pass
    
    # Try inline
    m = re.search(r'\{[^{}]*"name"\s*:\s*"([^"]+)"[^{}]*"arguments"\s*:\s*(\{[^{}]*\})', text)
    if m and m.group(1) in valid:
        return m.group(1), m.group(2)
    
    return None

# ═══════════════════════════════════════════════════════════════
# API Routes
# ═══════════════════════════════════════════════════════════════

@app.route('/v1/models', methods=['GET'])
def list_models():
    return jsonify({"object": "list", "data": [{"id": MODEL, "object": "model", "owned_by": "ollama"}]})

@app.route('/v1/chat/completions', methods=['POST'])
def chat():
    data = request.json
    messages = data.get('messages', [])
    tools = data.get('tools', [])
    valid_tools = get_valid_tools(tools)
    
    # Get last user message
    user_msg = next((m['content'] for m in reversed(messages) if m.get('role') == 'user' and m.get('content')), '')
    use_tools = needs_tools(str(user_msg))
    
    print(f"[{time.strftime('%H:%M:%S')}] '{str(user_msg)[:50]}' tools={use_tools}")
    
    # Add system prompt
    msgs = [{'role': 'system', 'content': SYSTEM_PROMPT}]
    msgs.extend(m for m in messages if m.get('role') != 'system')
    
    # Call Ollama
    try:
        payload = {'model': MODEL, 'messages': msgs, 'stream': False, 'options': {'num_ctx': 16384}}
        if use_tools and tools:
            payload['tools'] = tools
        
        r = req.post('http://localhost:11434/v1/chat/completions', json=payload, timeout=300)
        result = r.json()
        msg = result.get('choices', [{}])[0].get('message', {})
        content = msg.get('content', '')
        tool_calls = msg.get('tool_calls', [])
    except Exception as e:
        print(f"Error: {e}")
        content = "I'm ready to help! What would you like to code?"
        tool_calls = []
    
    # Return tool calls if present
    if tool_calls:
        print(f"  → Native tool calls")
        return jsonify({
            "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
            "object": "chat.completion",
            "created": int(time.time()),
            "model": MODEL,
            "choices": [{"index": 0, "message": {"role": "assistant", "content": None, "tool_calls": tool_calls}, "finish_reason": "tool_calls"}],
            "usage": {"prompt_tokens": 100, "completion_tokens": 100, "total_tokens": 200}
        })
    
    # Try to extract tool from text
    if use_tools and valid_tools and content:
        tool = extract_tool(content, valid_tools)
        if tool:
            name, args = tool
            print(f"  → Extracted tool: {name}")
            return jsonify({
                "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
                "object": "chat.completion",
                "created": int(time.time()),
                "model": MODEL,
                "choices": [{"index": 0, "message": {"role": "assistant", "content": None, "tool_calls": [{"id": f"call_{uuid.uuid4().hex[:8]}", "type": "function", "function": {"name": name, "arguments": args}}]}, "finish_reason": "tool_calls"}],
                "usage": {"prompt_tokens": 100, "completion_tokens": 100, "total_tokens": 200}
            })
    
    # Clean and return text
    if content:
        content = re.sub(r'```json\s*\{[^`]*"name"[^`]*\}\s*```', '', content).strip()
    if not content:
        content = "Hello! I'm ready to assist with your coding tasks."
    
    print(f"  → Text ({len(content)} chars)")
    return jsonify({
        "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
        "object": "chat.completion",
        "created": int(time.time()),
        "model": MODEL,
        "choices": [{"index": 0, "message": {"role": "assistant", "content": content}, "finish_reason": "stop"}],
        "usage": {"prompt_tokens": 100, "completion_tokens": len(content.split()), "total_tokens": 100 + len(content.split())}
    })

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "model": MODEL})

# Start server
threading.Thread(target=lambda: app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False), daemon=True).start()
time.sleep(2)
print(f'\n✅ API Server running with {MODEL}')

In [ ]:
#@title 🧪 Test API
import requests

tests = [
    ("Hi there!", "No tools"),
    ("What is Python?", "No tools"),
    ("Create a file test.py", "Should use tools"),
]

for msg, expected in tests:
    r = requests.post('http://localhost:5000/v1/chat/completions', json={
        'model': MODEL,
        'messages': [{'role': 'user', 'content': msg}],
        'tools': [{'type': 'function', 'function': {'name': 'write', 'parameters': {}}}]
    }, timeout=120)
    resp = r.json()['choices'][0]['message']
    has_tools = resp.get('tool_calls') is not None
    print(f"'{msg}' → {'🔧 Tools' if has_tools else '💬 Text'} ({expected})")

In [ ]:
#@title 🌐 Start Cloudflare Tunnel
import subprocess, re
from IPython.display import display, HTML

tunnel = subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in tunnel.stdout:
    print(line, end='')
    if 'trycloudflare.com' in line:
        m = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if m:
            url = m.group()
            display(HTML(f'''
            <div style="background:linear-gradient(135deg,#00b894,#00cec9);padding:30px;border-radius:20px;margin:20px 0">
                <h2 style="color:white;margin:0 0 15px 0">🚀 {MODEL} Ready!</h2>
                <p style="color:white;font-size:22px;font-family:monospace;background:rgba(0,0,0,0.2);padding:15px;border-radius:10px;margin:0">{url}/v1</p>
                <hr style="border:none;border-top:1px solid rgba(255,255,255,0.3);margin:20px 0">
                <p style="color:white;margin:5px 0"><b>OpenCode Setup:</b></p>
                <p style="color:#e0e0e0;margin:5px 0">Settings → Custom Provider → Base URL: <code>{url}/v1</code></p>
                <p style="color:#e0e0e0;margin:5px 0">Model: <code>{MODEL}</code></p>
            </div>
            '''))
            break

print('\n⚠️ Keep this cell running!')
for line in tunnel.stdout:
    print(line, end='')